In [13]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

normal_path = os.path.join("no")
tumor_path = os.path.join("yes")

def load_images_from_folder(folder, label, limit=100):
    images, labels = [], []
    for i, file in enumerate(os.listdir(folder)):
        if i >= limit:  
            break
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (224, 224))  
            img = preprocess_input(img) 
            images.append(img)
            labels.append(label)
    return images, labels

normal_images, normal_labels = load_images_from_folder(normal_path, label=0, limit=100)
tumor_images, tumor_labels = load_images_from_folder(tumor_path, label=1, limit=100)

X = np.array(normal_images + tumor_images)
y = np.array(normal_labels + tumor_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.15, height_shift_range=0.15,
                             zoom_range=0.15, horizontal_flip=True)
datagen.fit(X_train)

base_model = VGG16(weights="imagenet", include_top=False)  
selected_layers = ["block4_conv3", "block5_conv3"]  # Use fewer layers to reduce memory usage
layer_outputs = [base_model.get_layer(name).output for name in selected_layers]

feature_extraction_model = Model(inputs=base_model.input, outputs=layer_outputs)

def extract_hypercolumn_features(image):
    feature_maps = feature_extraction_model.predict(np.expand_dims(image, axis=0))
    hypercolumn = np.concatenate([GlobalAveragePooling2D()(fm).numpy().flatten() for fm in feature_maps])
    return hypercolumn

def extract_features_batch(X_data, batch_size=10):
    batch_features = []
    for i in range(0, len(X_data), batch_size):
        batch = X_data[i:i + batch_size]
        batch_features.extend([extract_hypercolumn_features(img) for img in batch])
    return np.array(batch_features)

X_train_hypercolumns = extract_features_batch(X_train, batch_size=10)
X_test_hypercolumns = extract_features_batch(X_test, batch_size=10)

# Feature Selection using Recursive Feature Elimination (RFE)
svm = SVC(kernel="linear")  
rfe = RFE(estimator=svm, n_features_to_select=200, step=10)  
X_train_selected = rfe.fit_transform(X_train_hypercolumns, y_train)
X_test_selected = rfe.transform(X_test_hypercolumns)

# Train SVM Classifier on Selected Features
svm.fit(X_train_selected, y_train)

# Make Predictions and Evaluate
y_pred = svm.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 613ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━

In [15]:
print(f"Final Classification Accuracy: {accuracy * 100:.2f}%")


Final Classification Accuracy: 80.00%
